In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1ddbfdbf4982d7a120fd38511f3531034adf37f366a6a9acf945bad509e889b6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('gouga').getOrCreate()

In [18]:
df_spark = spark.read.csv('data5.csv',header=True,inferSchema=True)
df_spark.show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|    amen| 23|         4|536987|
|   gouga| 23|         2|235689|
|   kouka| 20|         8|951236|
| ranouna| 21|         6|753214|
|   touta| 22|         8|987456|
|hachoumi| 23|         3|321587|
|  lo3mor| 23|         9|985124|
+--------+---+----------+------+



In [19]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [20]:
df_spark.columns

['Name', 'Age', 'Experience', 'Salary']

In [21]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['Age','Experience'],outputCol='independent_feature')

In [22]:
output = featureassembler.transform(df_spark)
output.show()

+--------+---+----------+------+-------------------+
|    Name|Age|Experience|Salary|independent_feature|
+--------+---+----------+------+-------------------+
|    amen| 23|         4|536987|         [23.0,4.0]|
|   gouga| 23|         2|235689|         [23.0,2.0]|
|   kouka| 20|         8|951236|         [20.0,8.0]|
| ranouna| 21|         6|753214|         [21.0,6.0]|
|   touta| 22|         8|987456|         [22.0,8.0]|
|hachoumi| 23|         3|321587|         [23.0,3.0]|
|  lo3mor| 23|         9|985124|         [23.0,9.0]|
+--------+---+----------+------+-------------------+



In [23]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'independent_feature']

In [24]:
finilized_data = output.select('independent_feature','Salary')
finilized_data.show()

+-------------------+------+
|independent_feature|Salary|
+-------------------+------+
|         [23.0,4.0]|536987|
|         [23.0,2.0]|235689|
|         [20.0,8.0]|951236|
|         [21.0,6.0]|753214|
|         [22.0,8.0]|987456|
|         [23.0,3.0]|321587|
|         [23.0,9.0]|985124|
+-------------------+------+



In [25]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finilized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='independent_feature',labelCol='Salary')
regressor=regressor.fit(train_data)

In [26]:
regressor.coefficients

DenseVector([41223.8554, 145477.988])

In [27]:
regressor.intercept

-1016380.9638551592

In [28]:
pred_results = regressor.evaluate(test_data)

In [29]:
pred_results.predictions.show()

+-------------------+------+------------------+
|independent_feature|Salary|        prediction|
+-------------------+------+------------------+
|         [22.0,8.0]|987456|1054367.7590361254|
|         [23.0,9.0]|985124|1241069.6024096028|
+-------------------+------+------------------+

